In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import tqdm.notebook as tqdm

OSError: [WinError 127] The specified procedure could not be found. Error loading "C:\Users\morara\anaconda3\lib\site-packages\torch\lib\torch_cuda.dll" or one of its dependencies.

In [44]:
class Discriminator(nn.Module):
    def __init__(self, img_dim):
        super().__init__()
        self.disc = nn.Sequential(
        nn.Linear(img_dim, 128),
        nn.LeakyReLU(0.1),
        nn.Linear(128, 1),
        nn.Sigmoid(),)
        
    def forward(self, x):
        return self.disc(x)
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        self.gen = nn.Sequential(
        nn.Linear(z_dim, 256),
        nn.LeakyReLU(0.1),
        nn.Linear(256, img_dim),
        nn.Tanh(),)
        
    def forward(self, x):
        return self.gen(x)

In [45]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 3e-4
z_dim = 64
img_dim = 28*28*1
batch_size = 32
num_epochs= 5

disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim, img_dim).to(device)
fixed_noise = torch.randn(batch_size, z_dim).to(device)
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5), (0.5))])


In [46]:
dataset = datasets.MNIST(root='dataset/', transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f'runs/GAN_MNIST/fake')
writer_real = SummaryWriter(f'runs/GAN_MNIST/real')
step = 0

In [47]:
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.view(-1, 784).to(device)
        bach_size = real.shape[0]
        
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = gen(noise)
        disc_real = disc(real).view(-1)
        lossD_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).view(-1)
        lossD_fake = criterion(disc_fake, torch.ones_like(disc_fake))
        lossD = (lossD_real + lossD_fake)/2
        disc.zero_grad()
        lossD.backward(retain_graph=True)
        opt_disc.step()
        
        output = disc(fake).view(-1)
        lossG = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        lossG.backward()
        opt_gen.step()
        
        if batch_idx ==0:
            print(
                f"Epoch [{epoch}/{num_epochs}]\n"
                f'Loss D: {lossD:.4f}, Loss G: {lossG: .4f}')
            with torch.no_grad():
                  fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
                  data = real.reshape(-1, 1, 28, 28)
                  img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
                  img_grid_real = torchvision.utils.make_grid(real, normalize=True)
                  
                  writer_fake.add_image('MNIST Fake Images', img_grid_fake, global_step=step)
                  writer_real.add_image('MNIST Real Images', img_grid_real, global_step=step)
                  
                  step +=1

Epoch [0/5]
Loss D: 0.6073, Loss G:  0.6484
Epoch [1/5]
Loss D: 0.0000, Loss G:  0.0000


KeyboardInterrupt: 